# !OUTDATED!
# Supported Type Casting

### Simple Support
The modeling database supports the input and output of all 'normal' native types.
```
native_types = ["int", "float", "bool", "list", "tuple", "dict", "set"]
```

The above type list is pretty easily supported and you shouldn't have any problems sticking these value types into the database and having them returned back to you when you use `modelingdbtools.query.get_dataset()`.

In [1]:
from modelingdbtools.utils import checks
import numpy

def decode_type(cast, var_in, **kwargs):
    checks.check_types(cast, str)
    checks.check_types(var_in, str)
    
    native_types = ["int", "float", "bool",
                    "list", "tuple", "dict", "set"]
    
    cast = cast.replace("<class '", "")[:-2]
    if "numpy.ndarray" in cast:
        var_in = var_in.replace("[", "")
        var_in = var_in.replace("]", "")
        result = numpy.fromstring(var_in, sep=" ", **kwargs)
    elif "str" in cast:
        result = eval(cast + "('{v}')".format(v=var_in))
    elif cast in native_types:
        result = eval(cast + "({v})".format(v=var_in))
    else:
        result = var_in
    
    return result

In [2]:
def test_decode(inp, **kwargs):
    print("in:\t", type(inp), inp)
    if not isinstance(inp, numpy.ndarray):
        out = decode_type(str(type(inp)), str(inp), **kwargs)
    else:
        out = decode_type(str(type(inp)), numpy.array_str(inp, precision=24), **kwargs)
    print("out:\t", type(out), out)
    print("Types match:", inp == out)

In [3]:
test_decode(5)

in:	 <class 'int'> 5
out:	 <class 'int'> 5
Types match: True


In [4]:
test_decode(12.33561)

in:	 <class 'float'> 12.33561
out:	 <class 'float'> 12.33561
Types match: True


In [5]:
test_decode("hello world")

in:	 <class 'str'> hello world
out:	 <class 'str'> hello world
Types match: True


In [6]:
test_decode(False)

in:	 <class 'bool'> False
out:	 <class 'bool'> False
Types match: True


In [7]:
test_decode(["hello world", 1, False, 42.00000003])

in:	 <class 'list'> ['hello world', 1, False, 42.00000003]
out:	 <class 'list'> ['hello world', 1, False, 42.00000003]
Types match: True


In [8]:
test_decode(tuple(["foo", "bar"]))

in:	 <class 'tuple'> ('foo', 'bar')
out:	 <class 'tuple'> ('foo', 'bar')
Types match: True


In [9]:
test_decode({"name": "jackson",
             "age": 22,
             "goodDev": True,
             "musicSelections": ["Car Seat Headrest"]})

in:	 <class 'dict'> {'name': 'jackson', 'age': 22, 'goodDev': True, 'musicSelections': ['Car Seat Headrest']}
out:	 <class 'dict'> {'name': 'jackson', 'age': 22, 'goodDev': True, 'musicSelections': ['Car Seat Headrest']}
Types match: True


In [10]:
test_decode(set([1, 2, 3, 5, 8, 13]))

in:	 <class 'set'> {1, 2, 3, 5, 8, 13}
out:	 <class 'set'> {1, 2, 3, 5, 8, 13}
Types match: True


### Difficult Support

While I want to say that the database easily supports the ingestion and querying of `numpy.ndarray`s as a value type, I cannot guarentee that it will. There is currently no support for ndarrays larger than 1 dimension due to a limit of [`numpy.fromstring`](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.fromstring.html). Additionally, `numpy.float64` are pretty heavily tested and have worked every time however I am having trouble with other types such as `numpy.bool_`, in such case I would recommend storing the ndarray as a the integer value of bools (0/ 1) and post processing them when you get them out.

In the case you want to store n-dim ndarrays, I have yet to attempt it but I think it would be possible to flattened ndarray in an `Iota`, and store an additional `Iota` with information about the original construction of that ndarray.

I may at a later time attempt to detect ndarrays during dataset ingestion and do this flattening, storing info, and conversion for you but that is a todo item and not directly needed as of yet.

Examples of simple 1d ndarrays and 3d ndarrays below.

In [11]:
# float succeeds
test_decode(numpy.random.rand(3))

in:	 <class 'numpy.ndarray'> [0.12889036 0.43467954 0.23113538]
out:	 <class 'numpy.ndarray'> [0.12889036 0.43467954 0.23113538]
Types match: [ True  True  True]


In [12]:
# int succeeds
test_decode(numpy.random.randint(2, size=10), dtype=numpy.int_)

in:	 <class 'numpy.ndarray'> [0 0 1 1 1 1 0 1 0 1]
out:	 <class 'numpy.ndarray'> [0 0 1 1 1 1 0 1 0 1]
Types match: [ True  True  True  True  True  True  True  True  True  True]


In [13]:
# bool fails
test_decode(numpy.random.rand(3) > 0.5, dtype=numpy.bool_)

in:	 <class 'numpy.ndarray'> [ True  True  True]
out:	 <class 'numpy.ndarray'> [ True]
Types match: [ True  True  True]


In [14]:
# n-dim fails
test_decode(numpy.random.rand(3, 2))

in:	 <class 'numpy.ndarray'> [[0.48622594 0.70521463]
 [0.13451223 0.74595621]
 [0.8358232  0.52357893]]
out:	 <class 'numpy.ndarray'> [0.48622594 0.70521463 0.13451223 0.74595621 0.8358232  0.52357893]
Types match: False


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  
